In [1]:
# imports

import copy
import random as rd
import statistics as st

In [2]:
# unit profiles

class FeralWarpwolf:
    def __init__(self, str_buff=0, mat_buff=0, arm_buff=0):
        # base attributes
        self.base_speed = 6
        self.base_strength = 11
        self.base_mat = 7
        self.base_defense = 14
        self.base_armor = 16
        self.base_fury = 4
        self.base_threshold = 9
        self.base_hp = 30
        self.base_cost = 16
        
        # base boxes
        self.mind1 = 4
        self.mind2 = 3
        self.mind3 = 2
        self.body1 = 5
        self.body2 = 4
        self.body3 = 2
        self.spirit1 = 4
        self.spirit2 = 4
        self.spirit3 = 2
        
        self.mind_disabled = False
        self.body_disabled = False
        self.spirit_disabled = False
        
        # Body: When this aspect is crippled, you will roll one less die on ALL Damage rolls this model makes.
        # Mind: When this aspect is crippled, you will roll one less die on ALL Attack rolls with this model.
        # Spirit: When this aspect is crippled, the model may not be forced, so it will not be able to generate Fury.
        
        # buffs
        self.str_buff = str_buff
        self.mat_buff = mat_buff
        self.arm_buff = arm_buff
        self.prot_plates = 0
        
        # current attributes
        self.speed = self.base_speed
        self.strength = self.base_strength + self.str_buff
        self.mat = self.base_mat + self.mat_buff
        self.defense = self.base_defense
        self.armor = lambda: self.base_armor + self.arm_buff + self.prot_plates
        self.fury = self.base_fury
        self.threshold = self.base_threshold
        self.hp = self.base_hp # this needs to be lambda: boxes
        self.cost = self.base_cost
        
        # weapons
        self.weapons = {
            'Bite': {
                'p+s': 3 + self.strength,
                'rof': 1
            },
            'Claw': {
                'p+s': 4 + self.strength,
                'rof': 2
            }
        }
        
    def protective_plates(self, active=False):
        if active:
            self.prot_plates = 2
        else:
            self.prot_plates = 0
            
        
class TitanGladiator:
    def __init__(self, str_buff=0, mat_buff=0, arm_buff=0):
        # base attributes
        self.base_speed = 4
        self.base_strength = 12
        self.base_mat = 6
        self.base_defense = 10
        self.base_armor = 19
        self.base_fury = 4
        self.base_threshold = 9
        self.base_hp = 30
        self.base_cost = 15
        
        # buffs
        self.str_buff = str_buff
        self.mat_buff = mat_buff
        self.arm_buff = arm_buff
        
        # current attributes
        self.speed = self.base_speed
        self.strength = self.base_strength + self.str_buff
        self.mat = self.base_mat + self.mat_buff
        self.defense = self.base_defense
        self.armor = lambda: self.base_armor + self.arm_buff
        self.fury = self.base_fury
        self.threshold = self.base_threshold
        self.hp = self.base_hp
        self.cost = self.base_cost
        
        self.weapons = {
            'Tusks': {
                'p+s': 3 + self.strength,
                'rof': 1
            },
            'War Gauntlet': {
                'p+s': 4 + self.strength,
                'rof': 2
            }
        }

In [3]:
asd = FeralWarpwolf()
print(asd.armor())
asd.protective_plates(True)
print(asd.armor())
asd.protective_plates(False)
print(asd.armor())

16
18
16


In [4]:
# functions

def roll_d6(num=1):
    """Rolls a specified number of six-sided dice."""
    
    return rd.randint(num,6)


def game_roll(bonus=0,boosted='No'):
    """Roll 2d6. Should be usable for ranged and melee attacks as well as damage."""
       
    first_result = roll_d6()
    second_result = roll_d6()
    total_result = first_result + second_result + bonus
    if boosted == 'Yes':
         total_result = total_result + roll_d6()
    special = ''
    
    if first_result == 1 and second_result == 1:
        special = 'Failure!'
    elif first_result==second_result:
        special = 'Crit!'
    
    # print(total_result, '(', first_result, '+', second_result, ')', special)
    
    return total_result


def melee_attack_roll(mat, defense, boosted='No'):
    result = game_roll(mat, boosted)
    
    if result >= defense:
        # print('Hit!')
        return 'Hit!'
    else:
        # print('Miss!')
        return 'Miss!'

    
def damage_roll(ps, armor, boosted='No'):
    result = game_roll(ps, boosted)
    
#     print('Result:', type(result))
#     print('Armor:', type(armor))
    
    if result > armor:
        damage = result - armor
    else:
        damage = 0
    
    # print(damage)
    return damage


def is_dead(unit):
    if unit.hp <= 0:
        return True


def initial_melee_attacks(attacker, defender):
    """This *should* just do the initial melee attacks a single time.
    Currently broken.
    """
    
    log = {
        'Total Damage': 0
    }
    
    for weapon, values in attacker.weapons.items():
        for attacks in range(values['rof']):
            result = melee_attack_roll(attacker.mat, defender.defense)
#             print(weapon, result)
            
            if result == 'Hit!':
                damage = damage_roll(values['p+s'], defender.armor())
                defender.hp = defender.hp - damage
                log['Total Damage'] += damage
#                 print('Damage:', damage, 'HP:', defender.hp)
            else:
                continue
            
            if is_dead(defender):
                return log
    return log


def rounds_to_kill(attacker, defender):
    """Returns the mean number of rounds it would take the attacker to kill the defender.
    Assumes the defender takes no actions.
    
    TODO: return a dictionary of lists
    """
    
    x = []
    
    for _ in range(10000):
        a = copy.copy(attacker)
        d = copy.copy(defender)
    
        log = {
            'Rounds': 0,
            'Total Damage': 0
        }
        
        while not is_dead(d):
            log['Rounds'] += 1
            log['Total Damage'] += initial_melee_attacks(a,d)['Total Damage']
            
        x.append(log['Rounds'])
    
    return st.mean(x)


def damage_per_round(attacker, defender):
    """text"""
    
    x = []
    
    for _ in range(10):
        a = copy.copy(attacker)
        d = copy.copy(defender)
        
        x.append(initial_melee_attacks(a, d)['Total Damage'])
    
    return st.mean(x)

In [5]:
# research suites

def standard_output(a, d):
    output = {
        'Damage per round': damage_per_round(a,d),
        'Rounds to kill': rounds_to_kill(a,d)
    }
    
    return output

In [6]:
# workbench


In [7]:
# laboratory

x = FeralWarpwolf(str_buff=0, mat_buff=0)
y = FeralWarpwolf(str_buff=0, mat_buff=0)

xy = standard_output(x,y)
yx = standard_output(y,x)

print(xy)
print(yx)

{'Damage per round': 9.8, 'Rounds to kill': 3.6434}
{'Damage per round': 9.9, 'Rounds to kill': 3.6581}


In [8]:
# todo

## warbeast damage tracks (mind, body, spirit)
## chance to hit (suite: where to spend boost)
## new_unit class that populates from unit dicts
## if attack: or if hit: true instead of 'Hit!'
## crit interactions with boosted